In [0]:
# input files: test dataset or whole dataset,paragraphs file
# ouput: Zip file containing testing dataset with selected paragraph

In [0]:
'''!pip install tensorflow==2.0
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece'''

'!pip install tensorflow==2.0\n!pip install tensorflow_hub\n!pip install bert-for-tf2\n!pip install sentencepiece'

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.0.0
Hub version:  0.8.0


In [0]:
import bert
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
#import math
import numpy as np
import zipfile
import json
import os
import glob
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
basePath = '/content/'

'''data_dir = basePath+'dataset'
drive_dir = basePath+'drive/My Drive/temp/DatasetN'
paraFile = basePath + 'paragraph2.json' '''

data_dir = basePath+'tempdataset'
drive_dir = basePath+'drive/My Drive/temp/tempDataset'
paraFile = basePath + 'temp.json'

outDir = basePath+'test'

max_seq_length = 512

In [0]:
'''for file_name in os.listdir(drive_dir):
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:
            zip_dir.extractall(path='/content/')'''

"for file_name in os.listdir(drive_dir):\n    if file_name.endswith('.zip'):\n        with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:\n            zip_dir.extractall(path='/content/')"

In [0]:
def getBERTModel():
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="segment_ids")

    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)
    #bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1",trainable=False)
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    FullTokenizer = bert.bert_tokenization.FullTokenizer
    tokenizer = FullTokenizer(vocab_file, do_lower_case)

    return {'model':model,'tokenizer':tokenizer}

In [0]:
def get_masks(tokens, max_seq_length):
    #print('len(tokens),max_seq_length)
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [0]:
def getEmbeddings(model,tokenizer,sentence): 
    stokens = tokenizer.tokenize(sentence)

    if len(stokens) > (max_seq_length - 2):
      stokens = stokens[:max_seq_length-2]

    stokens = ["[CLS]"] + stokens + ["[SEP]"]

    input_ids = get_ids(stokens, tokenizer, max_seq_length)
    #print(len(input_ids))
    input_masks = get_masks(stokens, max_seq_length)
    input_segments = get_segments(stokens, max_seq_length)

    '''print(input_masks)
    print(input_segments)'''

    pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])
    '''print('see')
    print(all_embs.shape)
    print(pool_embs.shape)'''
    # pool_ebmbs is an embeding of CLS token
    # all_embs contains embeding for words of input sentence.
    return pool_embs

In [0]:
def BERTEmbeddings(model,tokenizer,nSentences):
    trainX = np.asarray([])
    n = len(nSentences)
    for i in range(n): 
        if i%500 == 0:
          print( 'Processing ',i,' out of ',n)

        #senLen = len(nSentences[i].split())
        embs = getEmbeddings(model,tokenizer,nSentences[i])
        '''print(embs.shape)
        x = embs'''
        if trainX.shape[0] == 0:
            trainX = embs
        else:
            trainX = np.concatenate((trainX, embs), axis=0)
    return trainX

In [0]:
def getFileSent(inpFile):
    fileSent = ''
    article = ''
    with open(inpFile,'r') as f:
        x = json.loads(f.read())
        article = x['article']
        for option in x['options'][0]:
            fileSent += option
            fileSent += ' '
        fileSent += x['questions'][0]
    return fileSent,article

In [0]:
def getQuesEmb(model,tokenizer,inpFile):
    fileSent,article = getFileSent(inpFile)
    fileSent = fileSent + (' ' + fileSent)*5
    emb = getEmbeddings(model,tokenizer,fileSent)
    return emb,article

In [0]:
def getMatchingPara(quesEmb,paraEmbs):
    #paraEmbs = np.loadtxt('paraEmbs.csv',delimiter=',')

    scores = []
    for paraEmb in paraEmbs:
        score=float( cosine_similarity([paraEmb],[quesEmb]) )
        scores.append(score)
    print(scores)
    mxScoreInd = scores.index(max(scores))
    print(mxScoreInd)
    mxScoreInd += 1

    with open(paraFile,'r') as f:
      x = json.loads(f.read())
      return x['para'+str(mxScoreInd)]

In [0]:
def saveFile(inpFile,outFile,para):
    with open(inpFile,'r') as f:
      x = json.loads(f.read())
      x['article'] = para

      with open(outFile, 'w') as jsonOut:
          json.dump(x, jsonOut)

# **Embedding Generation**

In [0]:
outModel = getBERTModel()
model = outModel['model']
tokenizer = outModel['tokenizer']

In [0]:
f = open(paraFile,'r')
x = json.loads(f.read())
f.close()
fileSents = [ x['para'+str(id)] for id in range(1,len(x)+1)]
#print(len(fileSents))

In [0]:
paraEembs = BERTEmbeddings(model,tokenizer,fileSents)
print(paraEembs.shape)

Processing  0  out of  8
(8, 768)


In [0]:
np.savetxt(basePath+'paraEmbs.csv',paraEembs, delimiter=',',fmt='%.15f')

In [0]:
'''loaddata = np.loadtxt(basePath+'paraEmbs.csv', delimiter=',')
print(loaddata[0])'''

"loaddata = np.loadtxt(basePath+'paraEmbs.csv', delimiter=',')\nprint(loaddata[0])"

# **Generate Testing Dataset**

In [0]:
filenames = glob.glob(os.path.join(data_dir,'test')+"/*json")
if not os.path.exists(outDir):
    os.mkdir(outDir)
count = 0
for filename in filenames:
    emb,article = getQuesEmb(model,tokenizer,filename)
    print()
    para = getMatchingPara(emb[0],paraEembs)
    #print()
    print(article)
    print(para)
    if article == para:
        count += 1
    name = filename.split('/')[-1]
    #print(name)
    saveFile(filename,os.path.join(outDir,name),para)

print()
print('accuracy: ',(count/len(filenames))*100)


[0.6284343004226685, 0.8486784100532532, 0.32978305220603943, 0.9308595657348633, 0.7500879764556885, 0.9169883728027344, 0.8644481897354126, 0.9163175821304321]
3
But transgenic animals just have one novel gene. What about an animal with a whole new genome? Could a clone , a genetically exact copy of an organism, be developed using techniques associated with biotechnology? It could be argued that human cloning is one of the inevitable outcomes of modern biotechnology. It "simply" involves the removal of the nucleus from a somatic cell and its placement into an unfertilized egg cell whose nucleus has either been deactivated or removed. This new cell would mimic the zygote, the first diploid cell of a new organism. This new zygote is allowed to become established, and a few days later is placed into the uterus of a surrogate mother. Theoretically this would result in an individual genetically identical to the donor. Obviously, there are many ethical and legal issues associated with hum

In [0]:
with zipfile.ZipFile(basePath+'testdata.zip','w') as zf:
    for dirname, subdirs, files in os.walk(outDir):
        for filename in files:
            zf.write(os.path.join(dirname.split('/')[-1], filename))